# Download the dataset

In [0]:
!curl -L -o five_echo_NIH.tar.xz https://osf.io/ea5v3/download
!mkdir e5_data
!tar xvf five_echo_NIH.tar.xz -C e5_data/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   459  100   459    0     0    987      0 --:--:-- --:--:-- --:--:--   987
100 68.5M  100 68.5M    0     0  20.5M      0  0:00:03  0:00:03 --:--:-- 37.5M
p06.SBJ01_S09_Task11_e1.sm.nii.gz
p06.SBJ01_S09_Task11_e2.sm.nii.gz
p06.SBJ01_S09_Task11_e3.sm.nii.gz
p06.SBJ01_S09_Task11_e4.sm.nii.gz
p06.SBJ01_S09_Task11_e5.sm.nii.gz


#Install necessary packages

In [0]:
!pip install tedana

    100% |████████████████████████████████| 1.6MB 13.9MB/s 
    100% |████████████████████████████████| 2.3MB 9.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/00/49/3c/e1bab1714c0b83a21ea340bd150957dd90b8afb7617de4d6ca
Successfully built tedana


In [0]:
!ls -l

total 70160
drwxr-xr-x 2 root root     4096 Mar  3 09:30 e5_data
-rw-r--r-- 1 root root 71831808 Mar  3 09:29 five_echo_NIH.tar.xz
drwxr-xr-x 1 root root     4096 Feb 26 17:33 sample_data


# Let's run the tedana workflow

In [0]:
!tedana -d e5_data/p06.SBJ01_S09_Task11_e1.sm.nii.gz e5_data/p06.SBJ01_S09_Task11_e2.sm.nii.gz e5_data/p06.SBJ01_S09_Task11_e3.sm.nii.gz e5_data/p06.SBJ01_S09_Task11_e4.sm.nii.gz e5_data/p06.SBJ01_S09_Task11_e5.sm.nii.gz -e 15.4 29.7 44.0 58.3 72.6 --verbose --tedpca mle

Module `duecredit` not successfully imported due to "No module named 'duecredit'". Package functionality unaffected.
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
INFO:tedana.workflows.tedana:Using output directory: /content
INFO:tedana.workflows.tedana:Loading input data: ['e5_data/p06.SBJ01_S09_Task11_e1.sm.nii.gz', 'e5_data/p06.SBJ01_S09_Task11_e2.sm.nii.gz', 'e5_data/p06.SBJ01_S09_Task11_e3.sm.nii.gz', 'e5_data/p06.SBJ01_S09_Task11_e4.sm.nii.gz', 'e5_data/p06.SBJ01_S09_Task11_e5.sm.nii.gz']
INFO:tedana.workflows.tedana:Computing adaptive mask
INFO:tedana.workflows.tedana:Computing T2* map
INFO:tedana.combine:Optimally combining data with voxel-wise T2 estimates
INFO:tedana.decomposition.eigendecomp:Computing PCA of optimally combined multi-echo data
INFO:tedana.decomposition.eigendecomp:Making initial component selection guess from PCA results
INFO:te

In [0]:
%matplotlib inline
import os.path as op
from glob import glob

import numpy as np
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from nilearn import plotting

In [0]:
data_dir = 'e5_data/'
files = ['p06.SBJ01_S09_Task11_e1.sm.nii.gz',
         'p06.SBJ01_S09_Task11_e2.sm.nii.gz',
         'p06.SBJ01_S09_Task11_e3.sm.nii.gz',
         'p06.SBJ01_S09_Task11_e4.sm.nii.gz',
         'p06.SBJ01_S09_Task11_e5.sm.nii.gz']
files = [op.join(data_dir, f) for f in files]
imgs = [nib.load(f) for f in files]
echo_times = np.array([15.4, 29.7, 44.0, 58.3, 72.6])

n_echoes = len(imgs)
n_trs = imgs[0].shape[-1]

pal = sns.color_palette('cubehelix', n_echoes)

In [0]:
# Select voxel from around middle of brain
ts = [img.get_data()[22, 40, 2, :] for img in imgs]

# Prepare data for model
ts_1d = np.hstack(ts) # data raveled out (T * E)
log_data = np.log(np.abs(ts_1d) + 1)
# log_data = np.log(ts_1d)  # in a perfect world...
x = np.column_stack([np.ones(n_echoes), -1 * echo_times])
X = np.repeat(x, n_trs, axis=0)  # T * E

# Model fit
betas = np.linalg.lstsq(X, log_data, rcond=None)[0]
s0 = np.exp(betas[0])
r2s = betas[1]
t2s = 1. / r2s

# Values for plots
# Values from log-linear model
log_x = np.arange(-80, 0, .01)
log_y = betas[0] + log_x*betas[1]

# Values from monoexponential decay model
mono_x = np.arange(0, 80, .01)
mono_y = np.exp(-1*betas[1]*mono_x) * s0

# Get weights for optimal combination
alpha = echo_times * np.exp(-echo_times / t2s)
alpha = alpha / np.sum(alpha)  # unnecessary but good for bar plot below

# Combine data across echoes
oc = np.average(np.vstack(ts), axis=0, weights=alpha)

In [0]:
fig, axes = plt.subplots(n_echoes, sharex=True, sharey=False, figsize=(14, 6))
for i_echo in range(n_echoes):
    axes[i_echo].plot(ts[i_echo], color=pal[i_echo])
    axes[i_echo].set_ylabel('{0}ms'.format(echo_times[i_echo]), rotation=0, va='center', ha='right', fontsize=14)
    axes[i_echo].set_yticks([])
    axes[i_echo].set_xticks([])

axes[-1].set_xlabel('Time', fontsize=16)
axes[-1].set_xlim(0, len(ts[i_echo])-1)
fig.tight_layout()
fig.show()

In [0]:
fig, ax = plt.subplots(figsize=(10, 6))
values = [i[0] for i in ts]
for i_echo in range(n_echoes):
    rep_echo_times = np.ones(n_trs) * echo_times[i_echo]
    ax.scatter(rep_echo_times, ts[i_echo], alpha=0.05, color=pal[i_echo])

ax.set_ylabel('BOLD signal', fontsize=16)
ax.set_xlabel('Echo Time (ms)', fontsize=16)
ax.set_xticks(echo_times)
ax.tick_params(axis='both', which='major', labelsize=14)

ax.set_ylim(5000, 40000)
fig.tight_layout()
fig.show()

In [0]:
adaptive_mask_img = nib.load('adaptive_mask.nii')
fig, ax = plt.subplots(figsize=(10, 6))

plotting.plot_stat_map(adaptive_mask_img, vmax=8, alpha=1,
                       cut_coords=[0, -4, 6],
                       draw_cross=False, colorbar=True,
                       annotate=False, bg_img=None, figure=fig, axes=ax)
fig.show()

In [0]:
fig, ax = plt.subplots(figsize=(10, 6))
for i_echo in range(n_echoes):
    rep_echo_times = -1 * np.ones(n_trs) * echo_times[i_echo]
    log_echo_data = np.log((np.abs(ts[i_echo]) + 1))
    ax.scatter(rep_echo_times, log_echo_data, alpha=0.05, color=pal[i_echo])

ax.plot(log_x, log_y)

ax.set_ylabel('log(BOLD signal)', fontsize=16)
ax.set_xlabel('Negative Echo Time (ms)', fontsize=16)
ax.set_xticks(-1 * echo_times)
ax.set_xlim(-80, -10)
ax.set_ylim(8.6, 10.5)
ax.tick_params(axis='both', which='major', labelsize=14)

ax.annotate('$B_0$: {0:.02f}\n$B_1$: {1:.02f}'.format(betas[0], betas[1]),
            xy=(-70, 9.5), fontsize=16,
            bbox=dict(fc="white", ec="black", lw=1))

fig.tight_layout()
fig.show()

In [0]:
fig, ax = plt.subplots(figsize=(10, 6))
for i_echo in range(n_echoes):
    rep_echo_times = np.ones(n_trs) * echo_times[i_echo]
    ax.scatter(rep_echo_times, ts[i_echo], alpha=0.05, color=pal[i_echo])

ax.plot(mono_x, mono_y)

ax.set_ylabel('BOLD signal', fontsize=16)
ax.set_xlabel('Echo Time (ms)', fontsize=16)
ax.set_xticks(echo_times)
ax.set_xlim(10, 80)
ax.set_ylim(5000, 40000)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.annotate('$S_0$: {0:.02f}\n$T_2^*$: {1:.02f}'.format(s0, t2s),
            xy=(60, 20000), fontsize=16,
            bbox=dict(fc="white", ec="black", lw=1))

fig.tight_layout()
fig.show()

In [0]:
fig, ax = plt.subplots(figsize=(10, 6))
for i_echo in range(n_echoes):
    rep_echo_times = np.ones(n_trs) * echo_times[i_echo]
    ax.scatter(rep_echo_times, ts[i_echo], alpha=0.05, color=pal[i_echo])

ax.plot(mono_x, mono_y)

ax.axvline(t2s, 0, 1, label='$T_2^*$', color='black', linestyle='--', alpha=0.5)
ax.set_ylabel('BOLD signal', fontsize=16)
ax.set_xlabel('Echo Time (ms)', fontsize=16)
ax.set_xticks(np.hstack((echo_times, [np.round(t2s, 1)])))
ax.set_xlim(10, 80)
ax.set_ylim(5000, 40000)
ax.tick_params(axis='both', which='major', labelsize=14)

legend = ax.legend(frameon=True, fontsize=16)

fig.tight_layout()
fig.show()

In [0]:
fig, ax = plt.subplots()
sns.barplot(echo_times, alpha, ax=ax, palette=pal)
ax.set_ylabel('Weight', fontsize=16)
ax.set_xlabel('Echo Time (ms)', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=14)
fig.tight_layout()
fig.show()

# Optimally Combined Time series

In [0]:
fig, ax = plt.subplots(figsize=(10, 6))
for i_echo in range(n_echoes):
    rep_echo_times = np.ones(n_trs) * echo_times[i_echo]
    ax.scatter(rep_echo_times, ts[i_echo], alpha=0.05, color=pal[i_echo])

ax.plot(mono_x, mono_y)

# Optimal combination
rep_t2s = np.ones(n_trs) * t2s
ax.scatter(rep_t2s, oc, alpha=1, color='red', label='Optimally\ncombined\ndata')

ax.axvline(t2s, 0, 20000, label='$T_2^*$', color='black', linestyle='--', alpha=0.5)
ax.set_ylabel('BOLD signal', fontsize=16)
ax.set_xlabel('Echo Time (ms)', fontsize=16)
ax.set_xticks(np.hstack((echo_times, [np.round(t2s, 1)])))
ax.set_xlim(10, 80)
ax.set_ylim(5000, 40000)
ax.tick_params(axis='both', which='major', labelsize=14)

legend = ax.legend(frameon=True, fontsize=16)

fig.tight_layout()
fig.show()